In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset
import torchvision
from torchvision import models
from tqdm import tqdm
import pickle

/root/miniconda3/envs/CGAN/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/root/miniconda3/envs/CGAN/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [5]:
root = './../../data/cifar10'  # 数据集存储的根目录

# 定义数据集的变换
trainset = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2470, 0.2435, 0.2616)),
])

testset = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2470, 0.2435, 0.2616)),
])

# 加载数据集
train_set = datasets.CIFAR10(root, train=True, download=True, transform=trainset)
test_set = datasets.CIFAR10(root, train=False, download=True, transform=testset)

# 定义 DataLoader
batch_size = 50000
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

# 获取一个批次的数据
data_iter = iter(train_loader)
X_sub, _ = next(data_iter)
X_sub = X_sub.numpy()
print(X_sub)
print(len(X_sub))


Files already downloaded and verified
Files already downloaded and verified
[[[[-1.11625004e+00 -9.09851551e-01 -7.19330013e-01 ... -1.92596662e+00
    -1.91008973e+00 -1.89421296e+00]
   [-1.49729311e+00 -1.37027872e+00 -1.03686607e+00 ... -1.84658253e+00
    -1.83070576e+00 -1.94184339e+00]
   [-1.22738755e+00 -1.40203226e+00 -1.38615549e+00 ... -1.73544502e+00
    -1.62430739e+00 -1.86245942e+00]
   ...
   [-1.75132179e+00 -1.70369148e+00 -1.35440195e+00 ... -9.89235580e-01
    -1.49729311e+00 -1.56080031e+00]
   [-1.75132179e+00 -1.78307545e+00 -1.59255385e+00 ... -1.00511241e+00
    -1.46553946e+00 -1.60843062e+00]
   [-1.27501798e+00 -1.06861961e+00 -8.93974721e-01 ... -1.11625004e+00
    -1.49729311e+00 -1.67193782e+00]]

  [[-1.17503715e+00 -9.81777191e-01 -7.88517058e-01 ... -1.94807744e+00
    -1.94807744e+00 -1.94807744e+00]
   [-1.56155741e+00 -1.43271720e+00 -1.09451210e+00 ... -1.83534253e+00
    -1.83534253e+00 -1.94807744e+00]
   [-1.28777218e+00 -1.46492720e+00 -1.4488

In [6]:
root = './../../data/cifar100'  # 数据集存储的根目录

# 定义数据集的变换
trainset = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2470, 0.2435, 0.2616)),
])

testset = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2470, 0.2435, 0.2616)),
])

# 加载数据集
train_set = datasets.CIFAR100(root, train=True, download=True, transform=trainset)
test_set = datasets.CIFAR100(root, train=False, download=True, transform=testset)

# 定义 DataLoader
batch_size = 50000
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

# 获取一个批次的数据
data_iter1 = iter(train_loader)
X_sub1, _ = next(data_iter1)
X_sub1 = X_sub1.numpy()
print(X_sub1)
print(len(X_sub1))


Files already downloaded and verified
Files already downloaded and verified
[[[[ 0.77308893 -0.02075098  0.04275632 ... -1.11625    -0.84634435
    -0.71933   ]
   [ 1.1065016   0.7254585   0.5825674  ... -1.2432643  -1.2115108
    -1.1797571 ]
   [-0.2747797   0.64607453  0.5825674  ... -1.4972931  -1.4814163
    -1.6243074 ]
   ...
   [ 1.6621895   1.6621895   1.7098198  ...  1.8368342   1.852711
     1.8209574 ]
   [ 1.7574502   1.7415735   1.7892039  ...  1.8368342   1.8368342
     1.8844646 ]
   [ 1.8209574   1.7892039   1.852711   ...  1.852711    1.852711
     1.8209574 ]]

  [[ 0.88640344  0.2260983   0.2422033  ... -0.9656722  -0.6757821
    -0.466417  ]
   [ 1.2246085   0.87029845  0.80587846 ... -1.0945121  -1.0461972
    -0.9978822 ]
   [-0.01547689  0.90250844  0.82198346 ... -1.3038772  -1.2555622
    -1.4005072 ]
   ...
   [ 1.8365988   1.8365988   1.8527037  ...  1.7882837   1.8204937
     1.7882837 ]
   [ 1.8204937   1.8043886   1.8527037  ...  1.8527037   1.8527037
  

In [7]:
# 连接数组
concat_data = np.concatenate((X_sub, X_sub1), axis=0)
#print(concat_data)
print(len(concat_data))


100000


In [8]:
import numpy as np

# 加载数据
loaded_data = np.load('../../cifar/X_sub4500.npy')
print("原始数据长度:", len(loaded_data))
print("原始数据形状:", loaded_data.shape)

# 获取后1500个数据
last_1500_data = loaded_data[-4500:]
print("后1500个数据的长度:", len(last_1500_data))
print("后1500个数据的形状:", last_1500_data.shape)


原始数据长度: 9000
原始数据形状: (9000, 3, 32, 32)
后1500个数据的长度: 4500
后1500个数据的形状: (4500, 3, 32, 32)


In [9]:
# 连接数组
concat_data = np.concatenate((concat_data, last_1500_data), axis=0)
#print(concat_data)
print(len(concat_data))


104500


In [10]:
# 创建全零标签数组
labels_sub = np.zeros(concat_data.shape[0])

# 将前一半样本的标签设置为0，后一半样本的标签设置为1
#labels_sub[concat_data.shape[0] // 2:] = 1
#labels_sub[:150] = 0
#labels_sub[150:] = 1
# 将concat_data分成三个部分，并为每个部分赋予相应的标签
labels_sub[:50000] = 0  # 前1500个元素的标签为0
labels_sub[50000:100000] = 1  # 接下来的1500个元素的标签为1
labels_sub[100000:] = 2  # 最后1500个元素的标签为2
#labels_sub[4500:] = 3

# 打印标签数组的长度和0形状
print(len(labels_sub))
print(labels_sub.shape)

104500
(104500,)


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# 转换为PyTorch的Tensor对象
X_tensor = torch.from_numpy(concat_data).float()
y_tensor = torch.from_numpy(labels_sub).long()


# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)
print(X_train, X_test, y_train, y_test)
print("训练集长度:", len(X_train))
print("测试集长度:", len(X_test))
print("训练标签长度:", len(y_train))
print("测试标签长度:", len(y_test))
print(X_train.shape)
print(y_train.shape)

tensor([[[[ 2.0591e+00,  2.0591e+00,  2.0591e+00,  ...,  2.0591e+00,
            2.0591e+00,  2.0591e+00],
          [ 1.7257e+00,  1.3764e+00,  1.0112e+00,  ...,  1.0906e+00,
            1.0589e+00,  9.9536e-01],
          [ 8.3660e-01, -2.2715e-01, -1.5132e+00,  ..., -1.4179e+00,
           -1.4655e+00, -1.7354e+00],
          ...,
          [ 2.9679e-01,  1.2214e-01, -4.8742e-03,  ..., -1.9895e+00,
           -1.9895e+00, -1.9895e+00],
          [ 1.1065e+00,  9.9536e-01,  9.1598e-01,  ..., -2.9066e-01,
           -3.0653e-01, -3.2241e-01],
          [ 2.0591e+00,  2.0591e+00,  2.0591e+00,  ...,  2.0591e+00,
            2.0591e+00,  2.0591e+00]],

         [[ 2.1265e+00,  2.1265e+00,  2.1265e+00,  ...,  2.1265e+00,
            2.1265e+00,  2.1265e+00],
          [ 1.7722e+00,  1.4501e+00,  1.1280e+00,  ...,  1.0797e+00,
            1.0797e+00,  1.0152e+00],
          [ 8.7030e-01, -1.6042e-01, -1.3683e+00,  ..., -1.5293e+00,
           -1.5616e+00, -1.7709e+00],
          ...,
     

In [12]:
model = models.vgg16(pretrained=True)
num_classes = 3  # 假设要将类别数量修改为10
model.classifier[6] = torch.nn.Linear(4096, num_classes)

/root/miniconda3/envs/CGAN/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/envs/CGAN/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
# 定义逻辑回归模型
num_classes = 3
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# 将数据和标签送入设备（如GPU）
device = torch.device("cuda" if torch.cuda.is_available() else False)
X_train = X_train.to(device)
y_train = y_train.to(device)
model = model.to(device)
# 创建训练数据加载器
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# 创建测试数据加载器
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

# 假设您的模型已经定义为 model
# 假设您的数据加载器已经定义为 train_loader
# 假设您的设备已经定义为 device

num_epochs = 100
model.train()

# 使用 Adam 优化器和 L2 正则化
optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=1e-5)

# 交叉熵损失函数
criterion = nn.CrossEntropyLoss()

# 学习率调度器
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # 每100个批次打印一次
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.4f}")
            running_loss = 0.0

    # 更新学习率
    scheduler.step()
     # 每10个epoch保存一次模型
    if (epoch + 1) % 10 == 0:
        torch.save(model.state_dict(), f'./cifar/attackcifar_epoch{epoch+1}.pth')
# 保存训练好的模型参数

torch.save(model.state_dict(), './cifar/attackcifar5.pth')

print('训练完成')


NameError: name 'train_loader' is not defined

In [14]:
# 在测试集上评估模型性能
model.load_state_dict(torch.load('./cifar/attackcifar4.pth'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
test_loss = 0.0
correct = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)    
        outputs = model(inputs)
#       print(outputs)
        _, predicted = torch.max(outputs, 1)
#        print(predicted)
#        print(labels)
        if predicted == labels:
            correct += 1
    test_accuracy = correct / len(test_dataset)
print(f'Test Accuracy: {test_accuracy:.4f}')


Test Accuracy: 0.9132


In [20]:
# 初始化计数器
TP = 0
FP = 0
TN = 0
FN = 0
model.load_state_dict(torch.load('./cifar/attackcifar4.pth'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
# 在测试集上评估模型性能
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)    
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        # 更新计数器
        for label, prediction in zip(labels, predicted):
            if label == 0 and prediction == 0:
                TP += 1
            elif label != 0 and prediction == 0:
                FP += 1
            elif label == 0 and prediction != 0:
                FN += 1
            elif label != 0 and prediction != 0:
                TN += 1

# 计算FPR和FNR
FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
FNR = FN / (FN + TP) if (FN + TP) > 0 else 0

print(f"FPR: {FPR:.4f}")
print(f"FNR: {FNR:.4f}")


FPR: 0.0905
FNR: 0.0794


In [16]:
TP = 0  # 真阳性
FN = 0  # 假阴性
model.load_state_dict(torch.load('./cifar/attackcifar4.pth'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        for label, prediction in zip(labels, predicted):
            if label == 1 and prediction == 1:
                TP += 1  # 真阳性：正确预测为类别1
            elif label == 1 and prediction != 1:
                FN += 1  # 假阴性：错误预测为非类别1

# 计算 FNR
FNR = FN / (FN + TP) if (FN + TP) > 0 else 0
print(f"FNR for class 1: {FNR:.4f}")


FNR for class 1: 0.0903


In [21]:
root = './../../data/cifar10'  # 数据集存储的根目录

# 定义数据集的变换
trainset = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2470, 0.2435, 0.2616)),
])

testset = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2470, 0.2435, 0.2616)),
])

# 加载数据集
train_set = datasets.CIFAR10(root, train=True, download=True, transform=trainset)
test_set = datasets.CIFAR10(root, train=False, download=True, transform=testset)

# 定义 DataLoader
batch_size = 1
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

IN_count = 0
JBDA_count = 0
Knockoff_count = 0
input_count = 0

import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

model = models.vgg16(pretrained=True)
num_classes = 3  # 假设要将类别数量修改为10
model.classifier[6] = torch.nn.Linear(4096, num_classes)
model.load_state_dict(torch.load('./cifar/attackcifar4.pth'))
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# 在测试集上评估模型性能
model.eval()
with torch.no_grad():
    for inputs, _ in tqdm(test_loader, desc="Validation Progress"):
        input_count += 1
        inputs = inputs.to(device)
        outputs = model(inputs)
#        print(outputs)
        _, predicted = torch.max(outputs, 1)
        if predicted == 0 :
            IN_count += 1
        elif predicted == 1 :
            Knockoff_count += 1
        elif predicted == 2:
            JBDA_count += 1
    print(f"input: {input_count:}")
    print(f"IN: {IN_count:}")
    print(f"Knockoff Data: {Knockoff_count:}")
    print(f"JBDA Data: {JBDA_count:}")

Files already downloaded and verified
Files already downloaded and verified


/root/miniconda3/envs/CGAN/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/envs/CGAN/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Validation Progress: 100%|████████████████████████████████████| 10000/10000 [00:24<00:00, 415.94it/s]

input: 10000
IN: 8459
Knockoff Data: 1477
JBDA Data: 64


In [39]:
root = './../../data/cifar100'  # 数据集存储的根目录

# 定义数据集的变换
trainset = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2470, 0.2435, 0.2616)),
])

testset = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2470, 0.2435, 0.2616)),
])

# 加载数据集
train_set = datasets.CIFAR100(root, train=True, download=True, transform=trainset)
test_set = datasets.CIFAR100(root, train=False, download=True, transform=testset)

# 定义 DataLoader
batch_size = 1
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

IN_count = 0
JBDA_count = 0
Knockoff_count = 0
input_count = 0

import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

model = models.vgg16(pretrained=True)
num_classes = 3  # 假设要将类别数量修改为10
model.classifier[6] = torch.nn.Linear(4096, num_classes)
model.load_state_dict(torch.load('./cifar/attackcifar_epoch40.pth'))
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# 在测试集上评估模型性能
model.eval()
with torch.no_grad():
    for inputs, _ in tqdm(test_loader, desc="Validation Progress"):
        input_count += 1
        inputs = inputs.to(device)
        outputs = model(inputs)
#        print(outputs)
        _, predicted = torch.max(outputs, 1)
        if predicted == 0 :
            IN_count += 1
        elif predicted == 1 :
            Knockoff_count += 1
        elif predicted == 2:
            JBDA_count += 1
    print(f"input: {input_count:}")
    print(f"IN: {IN_count:}")
    print(f"Knockoff Data: {Knockoff_count:}")
    print(f"JBDA Data: {JBDA_count:}")

Files already downloaded and verified
Files already downloaded and verified


Validation Progress: 100%|████████████████████████████████████| 10000/10000 [00:24<00:00, 412.43it/s]

input: 10000
IN: 1679
Knockoff Data: 8282
JBDA Data: 39
